# DIPIKA GURTU 
## 2018IMT-029
## DM Lab Assignment-3
## Deadline: 10th December, 2021

## Objective:
Write a program in C/C++ or python to create a rule based classifier using indirect method of classification. The program should take a labeled dataset as input. Use the following dataset in a ratio of 70:30 for training and data.

Dataset: https://archive.ics.uci.edu/ml/datasets/HCV+data

Compute accuracy, precision, recall, and F1-score also.

In [138]:
import pandas as pd
import numpy as np
import random as rnd
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split

In [139]:
targetEntropy = 0

In [140]:
class treeNode():
    def __init__(self, col=-1, colName='', value=None, results=None, rb=None, lb=None):
        self.col = col 
        self.colName = colName; #name of the column the node represents
        self.value = value #value of the node
        self.results = results #results that are stored in the node
        self.rb = rb #the right children of the node
        self.lb= lb #the left children of the node

In [141]:
def partitionData(dataFrame, test_percentage):
    tot_index = range(len(dataFrame)) 
    test_indexes = rnd.sample(tot_index, int(test_percentage * len(dataFrame))) 
    train_indexes = list(set(tot_index) ^ set(test_indexes)) 

    # use the indexes generated to build up dataframes
    test_df = dataFrame.loc[test_indexes]
    train_df = dataFrame.loc[train_indexes]

    return train_df, test_df

In [142]:
def getUniqueClasses(dataFrameColumn):
    results = {}
    for row in dataFrameColumn:
        if row not in results: results[row] = 0 # initialise element in dictionary if not already there
        results[row] += 1 # increment value

    return results

In [143]:
def getEntropy(data, column):
    entropy = 0.0
    results = getUniqueClasses(data[column]) # get # of classes in data

    for row in results.values():
        p = float(row) / len(data[column]) # calculate probability value for each element in results
        entropy -= p * np.log2(p) # calculate entropy

    return entropy

In [144]:
def findSplitPoints(data, column):
    sorted = data.sort_values([column], ascending=True)
    sorted_matrix = sorted[[column, 'Category']].to_numpy()
    splitPoints = []
    previous = sorted_matrix[0][1] # get target of the first element in sorted matrix
    index = sorted.index.values; # get the indexes of each  element in the sorted matrix
    counter = 0
    for row in sorted_matrix:
        if row[1] != previous: 
            splitPoints.append([index[counter - 1], sorted_matrix[counter - 1][0]])
        counter += 1
        previous = row[1]

    return splitPoints

In [145]:
def splitSets(data, column, splitPoints):
    sets_below = []
    sets_above = []
    # split the dataframe into 2 for each splitpoint
    for i in range(len(splitPoints)):
        df1 = data[data[column] <= data[column][splitPoints[i][0]]]  # everything below the splitpoint
        df2 = data[data[column] > data[column][splitPoints[i][0]]]  # everything above it
        # add to the lists
        sets_below.append(df1)
        sets_above.append(df2)

    return sets_below, sets_above

In [146]:
def getInformationGain(data, column):
    splitpoints = findSplitPoints(data, column)  # get splitpoints for this column
    sets_below, sets_above = splitSets(data, column, splitpoints)  # split the data into sets based on these splitpoints
    # lists to store the # of instances in each subset that are above and below each given threshold and their entropies
    instances_above = []
    instances_below = []
    entropy_above = []
    entropy_below = []
    target_entropy = getEntropy(data, 'Category')  # get target entropy for the dataset
    # get entropy for sets above and below each of the thresholds
    for set in sets_below:
        entropy_below.append(getEntropy(set, 'Category'))
        instances_below.append(len(set))
    for set in sets_above:
        entropy_above.append(getEntropy(set, 'Category'))
        instances_above.append(len(set))

    totalInstances = []
    infoGains = []
    # work out the Information Gain for each threshold
    for i in range(len(instances_below)):
        totalInstances.append(instances_below[i] + instances_above[i])
        probA = (instances_above[i] / float(totalInstances[i]))
        probB = (instances_below[i] / float(totalInstances[i]))
        infoGains.append(target_entropy - ((entropy_below[i] * probB) + (entropy_above[i] * probA)))

    # work out the highest information gain for this column of the dataset
    best_gain = i = counter = 0
    for gain in infoGains:
        if best_gain < gain:
            best_gain = gain
            counter = i # variable to hold the index in the list where the best gain occurs
        i += 1

    return best_gain, sets_below[counter], sets_above[counter], splitpoints[counter]

In [147]:
def train(data):
    optimal_gain = -1
    best = {}
    columns = []
    i = 0

    for column in data:  # loop over each attribute
        if column != 'Category':
            try:
                ig, set1, set2, split = getInformationGain(data, column)  # get information gain for each column
                # column holds information that is used when creating a tree node.
                # the values in each of the columns will be used below when creating nodes for the tree
                columns.append({"ig": ig, "left": set1, "right": set2, 'col': i, 'split': split,'colName': column}) 

            # above code will work until the set1 and set2 values that would be returned bu the information Gain function will be 0
            # in that case, an indexError will be thrown as we cannot access element 0 of the sets lists
            # so if we catch that exception we know this data should be used as a leaf node and can format the tree information accordingly
            except IndexError:
                columns.append({"ig": 0, "left": [], "right": [], 'col': column, })
        i += 1  # counter to get int value for row(used for tree node)

    # loops through each column and pulls out the one with the best information gain for the given data
    for val in range(len(columns)):

        if columns[val]['ig'] > optimal_gain:
            best = columns[val]
            optimal_gain = columns[val]['ig']

    # get data for left branch and data for right branch
    left = best['left']
    right = best['right']
    # check if we have data for the left and right branches of the tree
    # if they are = 0 it is the stop condition for recursion, and the else block will generate a leaf node for the tree
    if len(best['left']) != 0 and len(best['right']) != 0:
        return (treeNode(col=best['col'], colName=best['colName'], value=best['split'][1], results=None,
                              rb=train(right), lb=train(left)))

    else:
        label = list(getUniqueClasses(data['Category']).keys()); #get label for the leaf node
        return (treeNode(results=label[0]))

In [148]:
def classify(target_row, tree):
    # base case to stop recursion -> we are at a leaf node
    if tree.results != None:
        return tree.results

    else:
        # gets the attribute from the target row that we are looking at
        val = target_row[tree.col]
        branch = None
        if isinstance(val, int) or isinstance(val, float):
            # checks the value of the tree against the value of the attribute from the target row
            # go down right side
            if val >= tree.value:
                branch = tree.rb
            # go down left side
            else:
                branch = tree.lb
        # recur over the tree again, using either the left or right branch to determine where to go next
        return classify(target_row, branch)

In [149]:
def printTree(tree, space=''):
    # Leaf node
    if tree.results != None:
        print(str(tree.results))

    else:
        print(str(tree.colName) + ' : ' + str(tree.value)) #name and splitpoint of current node
        print(space + 'L ', end="") #Print 'L' for left child
        printTree(tree.lb, space + ' ') #Print left child
        print(space + 'R ', end="") #Print 'R' for right child
        printTree(tree.rb, space + ' ') #Print right child

In [150]:
values = []
def test_tree(data, labels, tree):
   
    # Loop over each row in test data frame and get the classification result for each index
    for index, row in data.iterrows():
        values.append([index, classify(row, tree)])

    # Get the indexes from the test dataframe where each type occurs
    indexes = labels.index.values
    correct = incorrect = 0
    # Loop over values list and compare the class that was classified by the tree
    # and the class that was originally in the dataframe
    for l in range(len(values)):
        if values[l][0] == indexes[l] and values[l][1] == labels[indexes[l]]:
            correct += 1 #increment the correctly classified #
        else:
            incorrect += 1 #increment the incorrectly classified #

    return incorrect, correct, np.round(100 - (incorrect / (incorrect + correct)) * 100)

In [151]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [152]:
dataSet = pd.read_csv('/kaggle/input/hcvdat0/hcvdat0.csv')

In [153]:
dataSet.isnull().sum()

In [154]:
dataSet.drop('Unnamed: 0',axis='columns',inplace=True)

In [155]:
dataSet.dropna(inplace=True)

In [156]:
dataSet.head()

In [157]:
df=dataSet.Sex.map({'m':0,'f':1})

In [158]:
df.count()

In [159]:
dataSet.isnull().sum()

In [160]:
dataSet.info()

In [161]:
dataSet.Category.unique()

In [162]:
df1=dataSet.Category.map({'0=Blood Donor':1,'0s=suspect Blood Donor':2,'1=Hepatitis':3,'2=Fibrosis':4,'3=Cirrhosis':5})

In [163]:
df1.count()

In [164]:
dataSet.head()

In [165]:
dataSet.drop('Sex',axis='columns',inplace=True)

In [166]:
dataSet.drop('Category',axis='columns',inplace=True)

In [167]:
dataSet=pd.concat([df,dataSet,df1],axis=1)

In [168]:
dataSet.info()

In [169]:
dataSet.isnull().sum()

In [170]:
results = [];
tests = 1 
train_data, test_data = train_test_split(dataSet, test_size=0.3)
tree = train(train_data) # make tree
types = test_data['Category'] # get types column from test_data
        

incorrect, correct, accuracy = test_tree(test_data, types, tree) # test the tree
results.append(accuracy)

        # print information to console

print("Tree Generated:" + "\n")
printTree(tree)
print()
print("Correctly Classified: " + str(correct) + " / " + str(correct+incorrect))
print("Accuracy: " + str(accuracy))
print()

sum = 0
for r in range(len(results)):
    sum += results[r]
average = sum/tests

print("Average Accuracy after " + str(tests) + " runs")
print(average)
    #test_start
    
y_test=types
print(y_test)
    
vp=values
from pandas import DataFrame
df = DataFrame(values,columns=['index','pred_Target'])
app=df['pred_Target']
y_pred = app
print(y_pred)

from sklearn.metrics import classification_report, confusion_matrix,accuracy_score
print(confusion_matrix(y_test, y_pred))
print("Accuracy:",accuracy_score(y_test,y_pred)*100)
print(classification_report(y_test, y_pred))